In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [10]:
# Import data
data_path = '/Users/lukecullen/PycharmProjects/petrochemical-data/data/'
estimated_file = data_path+'combined/240325_Bio/facilityEmissions_bio.parquet'

electricity_mix_path = data_path+'extra_inputs/EF_electricity mix.xlsx'
start_year = 2020
end_year = 2050
gas = 'CO2e_100a'

In [11]:
# Current technology
estimated_emissions = pd.read_parquet(estimated_file)
estimated_emissions = estimated_emissions[estimated_emissions['Gas'] == gas]

# Electricity scenarios
def extract_from_electricity_scenarios(filepath, first_row, rows=32) -> pd.DataFrame:
    df = pd.read_excel(filepath, skiprows=first_row)[1:rows].reset_index(drop=True)
    df.drop(columns=df.columns[-2], inplace=True)
    df.rename(columns={df.columns[-1]:'Global_mean'}, inplace=True)
    df.columns = list(pd.Series(df.columns).str.upper().replace({'YEAR ':'YEAR', "COTE D'LVOIRE":'IVORY COAST', 'DRC':'CONGO (DEM.REP.)', 'KOREA':'SOUTH KOREA', 'RUSSIA':'RUSSIAN FEDERATION'}))
    return df.set_index('YEAR')

def impose_years_on_electricity_scenarios(df, bau=False, start_year:int=start_year, end_year:int=end_year, reference_year=2020):
    # Assume same as earliest year for propagation back
    end_propagation = df.index.min() if bau is False else end_year+1
    if reference_year not in df.index:
        print(reference_year + ' is not in dataframe so using '+ df.index.min()+' as reference')
        reference_year = df.index.min()

    for yr in range(start_year, end_propagation):
        df.loc[yr] = df.loc[reference_year]
    # Cut off years beyond end of required period
    return df[df.index <= end_year].sort_index()

policy_read, sustainable_read = [extract_from_electricity_scenarios(electricity_mix_path, first_row) for first_row in [5, 40]]

bau_scenario, policy_scenario, sustainable_scenario = [impose_years_on_electricity_scenarios(df, bau=bau) for df, bau in zip([policy_read.copy(), policy_read.copy(), sustainable_read.copy()], [True, False, False])]

# Currently all electricity conversions are using the GLO "electricity, medium voltage" value from EcoInvent. IEA EFs are CO2e_100yr so CO2e_100yr must be used here
current_elec_CF = 0.5565109131761606

bau_ratio, policy_ratio, sustainable_ratio = [df/current_elec_CF for df in [bau_scenario, policy_scenario, sustainable_scenario]]

In [12]:
pd.options.mode.chained_assignment = None 

def calculate_electricity_scenario_emissions(df:pd.DataFrame, ratio_df:pd.DataFrame, years:list, year_sigmas:list) -> pd.DataFrame:

    df_elec = df[df['Type']=='Electricity']
    df_indirect = df[df['Type']=='Indirect Utilities']

    total_elec, total_utl = pd.DataFrame(), pd.DataFrame()

    for country in np.sort(df_elec['COUNTRY/TERRITORY'].unique()):

        # Fetch appropriate entries for country
        country_emissions_elec = df_elec[df_elec['COUNTRY/TERRITORY']==country]
        country_emissions_utl = df_indirect[df_indirect['COUNTRY/TERRITORY']==country]

        # Subtract old electricity value from utilities
        country_emissions_utl[years] = country_emissions_utl[years].values-country_emissions_elec[years].values
        country_emissions_utl[year_sigmas] = country_emissions_utl[year_sigmas].values-country_emissions_elec[year_sigmas].values

        # Calculate new electricity values dependent on country ratio to used EI value
        ratios = ratio_df[country].values if country in list(ratio_df.columns[:-1]) else ratio_df['GLOBAL_MEAN'].values
        country_emissions_elec[years] = country_emissions_elec[years]*ratios
        country_emissions_elec[year_sigmas] = country_emissions_elec[year_sigmas]*ratios

        # Add new electricity values to utilities
        country_emissions_utl[years] = country_emissions_utl[years].values+country_emissions_elec[years].values
        country_emissions_utl[year_sigmas] = country_emissions_utl[year_sigmas].values+country_emissions_elec[year_sigmas].values

        total_elec, total_utl = pd.concat((total_elec, country_emissions_elec)), pd.concat((total_utl, country_emissions_utl))

    df_without = df[[i not in ['Electricity', 'Indirect Utilities'] for i in df['Type']]]

    return pd.concat((df_without, total_elec, total_utl))

years = list(map(str, list(range(start_year, end_year+1))))
year_sigmas = [year+'_sigma' for year in years]

estimated_emissions_bau = calculate_electricity_scenario_emissions(estimated_emissions, bau_ratio, years, year_sigmas)
estimated_emissions_policy = calculate_electricity_scenario_emissions(estimated_emissions, policy_ratio, years, year_sigmas)
estimated_emissions_sustainable = calculate_electricity_scenario_emissions(estimated_emissions, sustainable_ratio, years, year_sigmas)

In [25]:
estimated_emissions_policy[['Type']+years].groupby('Type').sum()

,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
Type,,,,,,,,,,,,,,,,,,,,,
Direct Process,64112.609869,81031.634376,82050.867505,93723.410989,104183.686633,111299.057258,116617.693835,119896.476783,123530.154136,126726.302094,...,185092.925369,186409.375939,187584.34899,188971.337163,190149.684719,191466.827071,192638.184077,193956.081752,195128.937641,196444.582884
Direct Utilities,91984.373868,96298.437376,98062.974871,101324.404328,104056.047948,106755.462024,110151.24608,112784.926289,115361.303558,117557.474257,...,136894.460114,138257.545078,139583.996095,140865.176846,142290.777244,143614.241864,144835.783781,146149.586147,147404.609899,148549.453937
EI & CM,1199648.247918,1256214.925316,1293015.965186,1338864.482248,1377525.146923,1417039.736696,1458072.495104,1498195.690066,1537214.887602,1570082.155542,...,1868577.323894,1889677.570185,1910609.818583,1931205.383245,1951997.490772,1971466.082233,1989238.646217,2008481.386906,2026372.517106,2042364.801387
Electricity,228586.908839,240140.761023,242134.644932,247004.1553,251300.371427,254769.944001,258132.590036,260173.308111,262474.680064,264035.955471,...,260083.897973,256856.085476,253609.571427,250365.177988,246657.84286,242812.621607,238353.167153,233949.728406,229192.120789,219489.014068
Feedstock,-388200.747532,-403296.344731,-420829.58028,-438729.380741,-451034.39912,-464473.433161,-480422.2848,-496824.116766,-513364.632376,-525029.548283,...,-626432.180972,-633693.266883,-640269.100551,-646921.809431,-653067.727088,-658431.26081,-663209.938057,-668279.031719,-672514.411177,-675564.073885
IHS CtOG,4301989.934497,4541607.258803,4611645.126016,4756726.457517,4897637.15779,5035713.802566,5162106.004555,5286639.946723,5410914.968933,5527156.331759,...,6624036.1171,6697857.96685,6779338.749145,6870808.94654,6948490.050492,7025296.325518,7101648.418552,7182048.650879,7259169.200799,7331963.809367
Indirect Utilities,1011611.030776,1065957.674991,1081205.613217,1111667.102232,1140966.806338,1168092.613068,1192412.938442,1216152.516468,1240206.296179,1261457.938942,...,1443904.621304,1453186.102712,1463573.287671,1474696.42377,1484176.307476,1493001.988807,1501099.433033,1510034.948422,1518092.859146,1520133.574734
Intermediates,1158836.186165,1227532.159281,1242688.243447,1276871.50565,1307111.624542,1342669.452243,1375596.42511,1407513.982866,1439411.622099,1469290.641857,...,1717534.122529,1738303.254714,1759086.548474,1779841.151608,1801279.986751,1821377.421947,1841733.556307,1862323.164087,1882458.36597,1902168.56738
N-fertilisers,2374.733328,2354.107491,2406.525021,2422.543731,2438.562442,2454.581152,2470.599863,2486.618573,2502.637284,2518.655994,...,2710.88052,2726.899231,2742.917941,2758.936652,2774.955362,2790.974073,2806.992783,2823.011494,2839.030204,2855.048915


In [29]:
estimated_emissions_sustainable[['Type']+years].groupby('Type').sum()

,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
Type,,,,,,,,,,,,,,,,,,,,,
Direct Process,6.411462e+04,8.103461e+04,8.205403e+04,9.368180e+04,1.039232e+05,1.110725e+05,1.164017e+05,1.196756e+05,1.233039e+05,1.264949e+05,...,1.847930e+05,1.861095e+05,1.872847e+05,1.886715e+05,1.898501e+05,1.911673e+05,1.923388e+05,1.936567e+05,1.948297e+05,1.961454e+05
Direct Utilities,9.395806e+04,9.842781e+04,1.002427e+05,1.035495e+05,1.063216e+05,1.090708e+05,1.125109e+05,1.151764e+05,1.177896e+05,1.200248e+05,...,1.398055e+05,1.412037e+05,1.425658e+05,1.438637e+05,1.453232e+05,1.466821e+05,1.479333e+05,1.492796e+05,1.505682e+05,1.517463e+05
EI & CM,6.657038e+05,7.002890e+05,7.202537e+05,7.453635e+05,7.669521e+05,7.891452e+05,8.111359e+05,8.321825e+05,8.527529e+05,8.714674e+05,...,1.042527e+06,1.055476e+06,1.068326e+06,1.081063e+06,1.093950e+06,1.106257e+06,1.117716e+06,1.129935e+06,1.141703e+06,1.152421e+06
Electricity,2.273240e+05,2.331372e+05,2.289396e+05,2.270427e+05,2.241845e+05,2.201137e+05,2.154051e+05,2.091338e+05,2.026096e+05,1.950025e+05,...,1.056212e+05,1.002567e+05,9.476976e+04,8.917501e+04,8.326221e+04,7.721005e+04,7.082342e+04,6.433004e+04,5.759523e+04,5.064745e+04
Feedstock,4.381545e+05,4.633060e+05,4.761675e+05,4.941352e+05,5.086832e+05,5.221682e+05,5.356440e+05,5.485473e+05,5.618295e+05,5.733209e+05,...,6.818252e+05,6.900173e+05,6.975697e+05,7.049734e+05,7.127306e+05,7.198692e+05,7.266473e+05,7.335746e+05,7.402748e+05,7.462523e+05
IHS CtOG,4.107905e+06,4.335608e+06,4.419450e+06,4.574510e+06,4.713905e+06,4.850430e+06,4.982991e+06,5.105530e+06,5.225242e+06,5.336326e+06,...,6.416556e+06,6.494461e+06,6.572717e+06,6.653748e+06,6.731338e+06,6.806679e+06,6.879620e+06,6.954461e+06,7.027248e+06,7.095005e+06
Indirect Utilities,1.006832e+06,1.054982e+06,1.063975e+06,1.087437e+06,1.109168e+06,1.128440e+06,1.144505e+06,1.159834e+06,1.174907e+06,1.186788e+06,...,1.281815e+06,1.288821e+06,1.296827e+06,1.305484e+06,1.312595e+06,1.319092e+06,1.325130e+06,1.331834e+06,1.337802e+06,1.342488e+06
Intermediates,9.518776e+05,1.004188e+06,1.013986e+06,1.041755e+06,1.066848e+06,1.095657e+06,1.122841e+06,1.149552e+06,1.176202e+06,1.201457e+06,...,1.422692e+06,1.441315e+06,1.459924e+06,1.478519e+06,1.497744e+06,1.515889e+06,1.534192e+06,1.552594e+06,1.570534e+06,1.588194e+06
N-fertilisers,2.374733e+03,2.354107e+03,2.406525e+03,2.422544e+03,2.438562e+03,2.454581e+03,2.470600e+03,2.486619e+03,2.502637e+03,2.518656e+03,...,2.710881e+03,2.726899e+03,2.742918e+03,2.758937e+03,2.774955e+03,2.790974e+03,2.806993e+03,2.823011e+03,2.839030e+03,2.855049e+03


In [31]:
estimated_emissions_bau[['Type']+years].groupby('Type').sum()

,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
Type,,,,,,,,,,,,,,,,,,,,,
Direct Process,64114.62477,81034.61245,82054.031918,93681.801773,103923.218247,111072.497224,116401.658995,119675.588193,123303.928974,126494.907423,...,184793.021828,186109.481945,187284.684559,188671.538627,189850.126283,191167.266054,192338.790903,193656.699636,194829.733281,196145.361591
Direct Utilities,93958.059222,98427.808587,100242.727996,103549.542379,106321.595145,109070.832539,112510.923219,115176.363031,117789.566013,120024.768202,...,139805.478179,141203.742937,142565.831346,143863.740131,145323.181542,146682.099882,147933.321895,149279.604187,150568.242954,151746.304189
EI & CM,665703.80897,700288.971427,720253.749944,745363.479151,766952.062087,789145.219685,811135.86868,832182.482256,852752.88942,871467.36236,...,1042527.394499,1055475.926444,1068325.587389,1081063.119164,1093950.439028,1106256.853199,1117716.294832,1129935.203945,1141702.506389,1152421.457784
Electricity,227324.025315,243293.776893,249835.172022,259670.652088,269267.118849,278399.161364,287714.655269,295924.877798,304669.384173,312989.776557,...,388769.667148,394209.663408,399870.536309,405843.569522,411665.660799,417185.191905,422342.802445,427885.357846,433054.574502,437769.179155
Feedstock,438154.529092,463306.012484,476167.536865,494135.168673,508683.228764,522168.212014,535644.036742,548547.332247,561829.519473,573320.925508,...,681825.239037,690017.276522,697569.714511,704973.416746,712730.640393,719869.164976,726647.347117,733574.648727,740274.788788,746252.299864
IHS CtOG,4107905.235387,4335607.711442,4419449.896706,4574510.348676,4713904.827532,4850429.663797,4982991.338862,5105530.297668,5225241.753051,5336326.190702,...,6416555.718724,6494461.414306,6572716.565865,6653747.683761,6731338.316659,6806678.743852,6879619.943805,6954460.801758,7027248.38395,7095004.744728
Indirect Utilities,1006831.853261,1065138.900808,1084870.184502,1120065.271089,1154250.463879,1186725.585468,1216814.962903,1246624.814837,1276966.90576,1304774.98882,...,1564963.442645,1582774.205616,1601927.599133,1622152.684781,1640998.558083,1659067.523382,1676649.68668,1695389.462171,1713261.035236,1729610.124803
Intermediates,951877.616082,1004188.325607,1013986.306633,1041755.479208,1066847.865992,1095656.818717,1122840.667313,1149551.995967,1176202.107337,1201456.72987,...,1422691.744185,1441314.70263,1459924.275836,1478518.531268,1497743.696334,1515888.750173,1534192.097128,1552593.754274,1570533.681856,1588194.365583
N-fertilisers,2374.733328,2354.107491,2406.525021,2422.543731,2438.562442,2454.581152,2470.599863,2486.618573,2502.637284,2518.655994,...,2710.88052,2726.899231,2742.917941,2758.936652,2774.955362,2790.974073,2806.992783,2823.011494,2839.030204,2855.048915


In [13]:
# Save to file
estimated_emissions_bau.to_parquet(data_path+'scenarios/ieu_bau_bio.parquet')
estimated_emissions_policy.to_parquet(data_path+'scenarios/ieu_ps_bio.parquet')
estimated_emissions_sustainable.to_parquet(data_path+'scenarios/ieu_ss_bio.parquet')